<a href="https://colab.research.google.com/github/mumin-ahmod/499-Grammar-Error-Detection/blob/main/GEC_LANG8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _,filenames in os.walk('kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [ ]:
!pip install happytransformer
import re
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
from happytransformer import HappyTextToText, TTSettings, TTTrainArgs
from transformers import T5Tokenizer

In [ ]:
#CONVERSION OF M2 FORMAT TO TEXT

In [ ]:
def main():
    m2 = open("/content/lang8.train.auto.bea19.m2").read().strip().split("\n\n")
    out = open("fce.train.gold.bea19.txt", "w")

    skip = {"noop", "UNK", "Um"}

    for sent in m2:
        sent = sent.split("\n")
        cor_sent = sent[0].split()[1:]
        edits = sent[1:]
        offset = 0
        for edit in edits:
            edit = edit.split("|||")
            if len(edit) < 3:
                continue  # Skip this edit if it doesn't have the expected format
            if edit[1] in skip:
                continue
            coder = int(edit[-1])
            if coder != 0:
                continue
            span = edit[0].split()[1:]
            start = int(span[0])
            end = int(span[1])
            cor = edit[2].split()
            cor_sent[start+offset:end+offset] = cor
            offset = offset-(end-start)+len(cor)
        out.write(" ".join(cor_sent)+"\n")

main()

In [ ]:
#PREPROCESSING INCORRECT SENTENCE

In [ ]:
file1 = open("/content/lang8.train.auto.bea19.m2", "r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
  temp = each_sent[i].split("\n")
  temp = temp[0]
  temp = temp.split(" ")
  temp = temp[1:]
  temp = ' '.join(temp)
  incorrect.append(temp)

**PREPROCESSING INCORRECT SENTENCE**


In [ ]:
file1 = open("/content/lang8.train.auto.bea19.m2", "r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
  temp = each_sent[i].split("\n")
  temp = temp[0]
  temp = temp.split(" ")
  temp = temp[1:]
  temp = ' '.join(temp)
  incorrect.append(temp)

**PREPROCESSING CORRECT SENTENCE**

In [ ]:
file2 = open("/content/fce.train.gold.bea19.txt", "r")
s2 = file2.read()
correct = s2.split("\n")

**STORING CORRECT AND INCORRECT SENTENCE IN DATAFRAME**

In [ ]:
df = pd.DataFrame()
df["target"] = correct
df["input"] = incorrect

**STORING IN CSV FORMAT**

In [ ]:
# df.to_csv("data.csv", index=False)
# index = []
# for i in range(len(df.values)):
#   if df.values[i][0] == df.values[i][1]:
#     index.append(i)

# df = df.drop(index)
# df.shape


# Save the DataFrame to a CSV file
df.to_csv("data.csv", index=False)

# Drop the rows where the first and second columns are the same
df = df[df.iloc[:, 0] != df.iloc[:, 1]]

# Print the shape of the modified DataFrame
print(df.shape)

(498360, 2)


**CHECKING NULL VALUES**

In [ ]:
df.isnull().values.any()

df.dropna(inplace=True)

**CHECKING DUPLICATE VALUES**

In [ ]:
df.duplicated().values.any()
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

**TEXT CLEANING**

In [ ]:
def clean(text):
  text = re.sub('<.*>', '', text)
  text = re.sub('\(.*\)', '',text)
  text = re.sub('\[.*]', '', text)
  text = re.sub('{.*}', '', text)
  text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
  text = text.replace("\\","")
  text = re.sub("\[","",text)
  text = re.sub("\]","",text)
  text = re.sub("[0-9]","",text)
  return text

**LOWERCASE CONVERSION**

In [ ]:
def word_cloud(df,col):
  comment_words = ''
  stopwords = set(STOPWORDS)

  for val in df[col]:
    val = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "

**SPLIT DATA INTO TRAINING AND EVALUATION**

In [ ]:
train_ratio = 0.8
train_size = int(train_ratio * len(df))
train_data = df[:train_size]
eval_data = df[train_size:]

train_data.to_csv("train.csv", index=False)
eval_data.to_csv("eval.csv", index=False)

**TRUNCATE AND LOAD TRAIN DATASET**

In [ ]:
df = pd.read_csv("train.csv")
desired_length = 10000
truncated_df = df[:desired_length]

truncated_df.to_csv("t_train.csv", index=False)

**TOKENIZATION**

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

input_sentences = train_data["input"].tolist()
input_encodings = tokenizer(input_sentences, padding=True, truncation=True, return_tensors="pt")

target_sentences = train_data["target"].tolist()
target_encodings = tokenizer(target_sentences, padding=True, truncation=True, return_tensors="pt")

train_data["input_ids"] = input_encodings.input_ids.tolist()
train_data["attention_mask"] = input_encodings.attention_mask.tolist()
train_data["target_ids"] = target_encodings.input_ids.tolist()
train_data["target_attention_mask"] = target_encodings.attention_mask.tolist()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**MODEL TRAINING**

In [ ]:
happy_tt = HappyTextToText("T5", "t5-base")

args = TTTrainArgs(batch_size=4)
happy_tt.train("t_train.csv", args=args)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1,1.025600,1.193385


**BEAM SEARCH ALGOROTIHM**

In [ ]:
def generator_text(input_text, happy_tt, tokenizer, beam_settings):
  input_text = "grammar: " + input_text
  result = happy_tt.generate_text(input_text, args=beam_settings)
  return result.text

beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)


**Example**

In [ ]:
example_1 = "my email id is abc@gmail.com"
result_1 = generate_text(example_1, happy_tt, tokenizer, beam_settings)
print(result_1)

In [ ]:
example_2 = "I am enjoys, writings articles ons AI and I also enjoy writings articles on AI"
result_2 = generate_text(example_2, happy_tt, tokenizer, beam_settings)
print(result_2)